In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
from sklearn.linear_model import LinearRegression

### Population Dataset

In [2]:
# Initialize Spark session 
spark = SparkSession.builder.appName("Databricks CSV Example").getOrCreate()

# Define the path to the CSV file in DBFS
file_path = "../Resources/population_census.csv"  

# Load the CSV file into a DataFrame
population_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame content
population_df.show()

# Print the DataFrame schema
population_df.printSchema()


your 131072x1 screen size is bogus. expect trouble
24/06/17 09:40:42 WARN Utils: Your hostname, DevanshNZ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/17 09:40:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/17 09:40:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+---------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|LGA code|Local Government Area|  2001|  2002|  2003|  2004|  2005|  2006|  2007|  2008|  2009|  2010|  2011|  2012|  2013|  2014|  2015|  2016|  2017|  2018|  2019|  2020|  2021|  2022|  2023|
+--------+---------------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+
|   10050|               Albury| 45265| 45816| 46180| 46505| 47004| 47566| 48140| 48518| 48814| 49222| 49451| 49785| 50338| 50990| 51486| 52171| 53056| 53922| 54657| 55466| 56067| 56665| 57517|
|   10180|             Armidale| 27906| 27774| 27610| 27410| 27350| 27377| 27468| 27788| 28156| 28409| 28573| 28748| 28895| 29015| 29160| 29310| 29519| 29631| 29701| 29600| 29332| 29361| 29594|
|   10250|              Ballin

In [3]:
#Filter dataset for specific columns
pop_df=population_df.select("LGA code", "Local Government Area", "2021", "2022", "2023")

In [4]:
#Remove rows with Nan values
pop_df= pop_df.dropna()

In [5]:
# Dictionary of old and new column names
rename_dict = {
    "2021": "Population in 2021",
    "2022": "Population in 2022",
    "2023": "Population in 2023"
}

# Initialize Renamed_Income_df with the original DataFrame
filterd_pop_df = pop_df

# Apply renaming using a loop
for old_name, new_name in rename_dict.items():
    filterd_pop_df = filterd_pop_df.withColumnRenamed(old_name, new_name)

# Show the final DataFrame with all renamed columns
filterd_pop_df.show()

+--------+---------------------+------------------+------------------+------------------+
|LGA code|Local Government Area|Population in 2021|Population in 2022|Population in 2023|
+--------+---------------------+------------------+------------------+------------------+
|   10050|               Albury|             56067|             56665|             57517|
|   10180|             Armidale|             29332|             29361|             29594|
|   10250|              Ballina|             46196|             46849|             47279|
|   10300|            Balranald|              2208|              2210|              2202|
|   10470|             Bathurst|             43674|             44110|             44612|
|   10500|        Bayside (NSW)|            176229|            177992|            182987|
|   10550|          Bega Valley|             35770|             35985|             36279|
|   10600|            Bellingen|             13203|             13261|             13231|
|   10650|

### Income Dataset

In [6]:
# Define the path to the CSV file in DBFS
file_path = "../Resources/Income_by_geography.csv"  

# Load the CSV file into a DataFrame
Income_df = spark.read.csv(file_path, header=True, inferSchema=True)

Income_df.show()

Income_df.printSchema()

+---------------+-----------------+-----------------+---------------------+-------------------+--------+--------+-------+-------+-------+-------+----------------+-------------------+-------------------+--------------------+---------------+---------------+--------------+----------------+
|            LGA|         LGA NAME|Number of Earners|Median age of earners|                Sum|  Median|    Mean|P80/P20|P80/P50|P20/P50|P10/P50|Gini coefficient|Income share:Top 1%|Income share:Top 5%|Income share:Top 10%|Lowest Quartile|Second Quartile|Third Quartile|Highest Quartile|
+---------------+-----------------+-----------------+---------------------+-------------------+--------+--------+-------+-------+-------+-------+----------------+-------------------+-------------------+--------------------+---------------+---------------+--------------+----------------+
|     Australia |             NULL|      14,760,008 |                  42 | 1,040,901,408,310 | 54,890 | 70,522 |  5.00 |  1.88 |  0.38 

In [7]:
#Filter dataset for specific columns
Filtered_Income_df=Income_df.select("LGA", "LGA NAME", "Number of Earners", "Median age of earners", "Mean")

In [8]:
# Dictionary of old and new column names
rename_dict = {
    "LGA": "LGA code",
    "LGA NAME": "Local Government Area",
    "Median age of earners": "Median Age of Earners",
    "Mean": "Mean Income"
}

# Initialize Renamed_Income_df with the original DataFrame
Renamed_Income_df = Filtered_Income_df

# Apply renaming using a loop
for old_name, new_name in rename_dict.items():
    Renamed_Income_df = Renamed_Income_df.withColumnRenamed(old_name, new_name)

# Show the final DataFrame with all renamed columns
Renamed_Income_df.show()


+---------------+---------------------+-----------------+---------------------+-----------+
|       LGA code|Local Government Area|Number of Earners|Median Age of Earners|Mean Income|
+---------------+---------------------+-----------------+---------------------+-----------+
|     Australia |                 NULL|      14,760,008 |                  42 |    70,522 |
|New South Wales|                 NULL|       4,603,736 |                  42 |    74,094 |
|          10050|               Albury|          31,816 |                  43 |    62,395 |
|          10180|    Armidale Regional|          15,535 |                  43 |    58,514 |
|          10250|              Ballina|          26,901 |                  48 |    60,261 |
|          10300|            Balranald|           1,316 |                  43 |    53,342 |
|          10470|    Bathurst Regional|          23,828 |                  43 |    64,364 |
|          10500|              Bayside|         105,143 |                  38 | 

In [9]:
Renamed_Income_df = Renamed_Income_df.dropna(subset=['Local Government Area'])
Renamed_Income_df.printSchema()


root
 |-- LGA code: string (nullable = true)
 |-- Local Government Area: string (nullable = true)
 |-- Number of Earners: string (nullable = true)
 |-- Median Age of Earners: string (nullable = true)
 |-- Mean Income: string (nullable = true)



In [10]:
# Clean the data: Remove non-numeric characters (commas, spaces) before casting
cleaned_Income_df = Renamed_Income_df.withColumn("Number of Earners", regexp_replace("Number of Earners", "[^0-9]", "")) \
                                     .withColumn("Mean Income", regexp_replace("Mean Income", "[^0-9]", ""))

# Cast cleaned columns to integer
cleaned_Income_df = cleaned_Income_df.withColumn("Number of Earners", col("Number of Earners").cast("int")) \
                                     .withColumn("Mean Income", col("Mean Income").cast("int"))

# Also cast 'LGA code' to integer, but ensure it only processes numeric values
cleaned_Income_df = cleaned_Income_df.withColumn("LGA code", col("LGA code").cast("int"))

# Show the resulting DataFrame
cleaned_Income_df.show()


+--------+---------------------+-----------------+---------------------+-----------+
|LGA code|Local Government Area|Number of Earners|Median Age of Earners|Mean Income|
+--------+---------------------+-----------------+---------------------+-----------+
|   10050|               Albury|            31816|                  43 |      62395|
|   10180|    Armidale Regional|            15535|                  43 |      58514|
|   10250|              Ballina|            26901|                  48 |      60261|
|   10300|            Balranald|             1316|                  43 |      53342|
|   10470|    Bathurst Regional|            23828|                  43 |      64364|
|   10500|              Bayside|           105143|                  38 |      67148|
|   10550|          Bega Valley|            20488|                  51 |      51652|
|   10600|            Bellingen|             7150|                  50 |      51346|
|   10650|             Berrigan|             4757|               

In [11]:
Renamed_Income_df = cleaned_Income_df

### Merging cleaned Population and Income datsets together

In [12]:
# Alias the DataFrames before joining
pop_alias = filterd_pop_df.alias("pop")
income_alias = Renamed_Income_df.alias("income")

# Join the aliased DataFrames
Pop_Inc_df = pop_alias.join(income_alias, 
                             (pop_alias['LGA code'] == income_alias['LGA code']) & 
                             (pop_alias['Local Government Area'] == income_alias['Local Government Area']), 
                             'inner')

# Select only the columns you want to show in the merged DataFrame
Pop_Inc_df = Pop_Inc_df.select("pop.LGA code", "pop.Local Government Area", "Population in 2021", 
                                "Population in 2022", "Population in 2023", 
                                "Number of Earners", "Median Age of Earners", "Mean Income")

# Show the merged DataFrame
Pop_Inc_df.show()

+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+
|LGA code|Local Government Area|Population in 2021|Population in 2022|Population in 2023|Number of Earners|Median Age of Earners|Mean Income|
+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+
|   10050|               Albury|             56067|             56665|             57517|            31816|                  43 |      62395|
|   10250|              Ballina|             46196|             46849|             47279|            26901|                  48 |      60261|
|   10300|            Balranald|              2208|              2210|              2202|             1316|                  43 |      53342|
|   10550|          Bega Valley|             35770|             35985|             36279|            20488|                  51 |      51652|
|   10

### Australian Postcode datset with Lat & Long

In [13]:
# Define the path to the CSV file in DBFS
file_path = "../Resources/australian_postcodes.csv"  

# Load the CSV file into a DataFrame
Postcode_df = spark.read.csv(file_path, header=True, inferSchema=True)

Postcode_df.show()

Postcode_df.printSchema()

24/06/17 09:41:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+--------+--------------------+-----+-----------+------------+--------------------+-----------------+--------------------+-----+--------------+----+--------------------+------+-----------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------+-------+-------+------------+--------+--------+----+----------------+----------+--------+------------------+--------------------+-------+----------+------------------+--------+-----------+
|   id|postcode|            locality|state|       long|         lat|                  dc|             type|              status|  sa3|       sa3name| sa4|             sa4name|region|Lat_precise|Long_precise|SA1_CODE_2021|SA1_NAME_2021|SA2_CODE_2021|SA2_NAME_2021|SA3_CODE_2021|SA3_NAME_2021|SA4_CODE_2021|SA4_NAME_2021|RA_2011|RA_2016|RA_2021|RA_2021_NAME|MMM_2015|MMM_2019| ced|        altitude|chargezone|phn_code|          phn_name|           lgaregion|lgacode|electorate|  elector

In [14]:
#Filter dataset for specific columns
Postcode_df = Postcode_df.select("postcode", "Lat_precise", "Long_precise", "lgaregion", "lgacode")

In [15]:
# Dictionary of old and new column names
rename_dict = {
    "postcode": "Postcode",
    "lgacode": "LGA code",
    "lgaregion": "Local Government Area",
}

# Initialize Renamed_Income_df with the original DataFrame
Renamed_Postcode_df = Postcode_df

# Apply renaming using a loop
for old_name, new_name in rename_dict.items():
    Renamed_Postcode_df = Renamed_Postcode_df.withColumnRenamed(old_name, new_name)

# Show the final DataFrame with all renamed columns
Renamed_Postcode_df.show()

+--------+-----------+------------+---------------------+--------+
|Postcode|Lat_precise|Long_precise|Local Government Area|LGA code|
+--------+-----------+------------+---------------------+--------+
|     200|   -35.2777|     149.119|   Unincorporated ACT|   89399|
|     200|-35.2776999|  149.118527|   Unincorporated ACT|   89399|
|     800|-12.3932794| 130.7766611| Darwin Waterfront...|   71150|
|     800|-12.3932794| 130.7766611| Darwin Waterfront...|   71150|
|     801|-12.4634403| 130.8456418|               Darwin|   71000|
|     803|        0.0|         0.0|                 NULL|    NULL|
|     804|-12.4324801| 130.8462536|               Darwin|   71000|
|     810|     -12.38|     130.873|               Darwin|   71000|
|     810|     -12.37|     130.867|               Darwin|   71000|
|     810| -12.373333|  130.881667|               Darwin|   71000|
|     810|   -12.3989|     130.852|               Darwin|   71000|
|     810| -12.386667|  130.875278|               Darwin|   71

In [16]:
Renamed_Postcode_df = Renamed_Postcode_df.dropna(subset=['Local Government Area'])
Renamed_Postcode_df.show()

+--------+-----------+------------+---------------------+--------+
|Postcode|Lat_precise|Long_precise|Local Government Area|LGA code|
+--------+-----------+------------+---------------------+--------+
|     200|   -35.2777|     149.119|   Unincorporated ACT|   89399|
|     200|-35.2776999|  149.118527|   Unincorporated ACT|   89399|
|     800|-12.3932794| 130.7766611| Darwin Waterfront...|   71150|
|     800|-12.3932794| 130.7766611| Darwin Waterfront...|   71150|
|     801|-12.4634403| 130.8456418|               Darwin|   71000|
|     804|-12.4324801| 130.8462536|               Darwin|   71000|
|     810|     -12.38|     130.873|               Darwin|   71000|
|     810|     -12.37|     130.867|               Darwin|   71000|
|     810| -12.373333|  130.881667|               Darwin|   71000|
|     810|   -12.3989|     130.852|               Darwin|   71000|
|     810| -12.386667|  130.875278|               Darwin|   71000|
|     810| -12.339444|  130.893056|               Darwin|   71

### Merging Australian Postcode dataset with Polulation and Income dataset

In [17]:
# Alias the DataFrames before joining
pop_alias = Pop_Inc_df.alias("pop")
pc_alias = Renamed_Postcode_df.alias("pc")

# Join the aliased DataFrames on both 'LGA code' and 'Local Government Area'
Pop_Inc_PC_df = pop_alias.join(pc_alias, 
                             (pop_alias['LGA code'] == pc_alias['LGA code']) & 
                             (pop_alias['Local Government Area'] == pc_alias['Local Government Area']), 
                             'inner')

# Select only the columns you want to show in the merged DataFrame
Pop_Inc_PC_df = Pop_Inc_PC_df.select("pc.Postcode", 
                                     "pop.LGA code", 
                                     "pop.Local Government Area", 
                                     "Population in 2021", 
                                     "Population in 2022", 
                                     "Population in 2023", 
                                     "Number of Earners", 
                                     "Median Age of Earners", 
                                     "Mean Income", 
                                     "pc.Lat_precise", 
                                     "pc.Long_precise")

# Show the merged DataFrame
Pop_Inc_PC_df.show()
Pop_Inc_PC_df.printSchema()


+--------+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+-----------+------------+
|Postcode|LGA code|Local Government Area|Population in 2021|Population in 2022|Population in 2023|Number of Earners|Median Age of Earners|Mean Income|Lat_precise|Long_precise|
+--------+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+-----------+------------+
|    3691|   10050|               Albury|             56067|             56665|             57517|            31816|                  43 |      62395|   -36.1241|     146.882|
|    3691|   10050|               Albury|             56067|             56665|             57517|            31816|                  43 |      62395|-36.4916755| 147.0710248|
|    3691|   10050|               Albury|             56067|             56665|             57517|            31816|    

### Vehicle Registration Dataset

In [18]:
# Define the path to the CSV file in DBFS
file_path = "../Resources/Postcode_registration.csv"  

# Load the CSV file into a DataFrame
EV_df = spark.read.csv(file_path, header=True, inferSchema=True)

EV_df.show()

EV_df.printSchema()

+--------+-----+-----------+-------------------------------------+-------------------------------------+-------------------------------------+
|Postcode|State|  Fuel Type| Registrations as at 31 January 2021 | Registrations as at 31 January 2022 | Registrations as at 31 January 2023 |
+--------+-----+-----------+-------------------------------------+-------------------------------------+-------------------------------------+
|     800|   NT|        ICE|                               6,780 |                               7,318 |                               7,089 |
|     800|   NT|        BEV|                                   3 |                                   3 |                                  12 |
|     800|   NT|      HFCEV|                                 -   |                                 -   |                                 -   |
|     800|   NT|Hybrid/PHEV|                                 131 |                                 411 |                                 402 |

In [19]:
# Rename columns with special characters
EV_df = EV_df.withColumnRenamed(" Registrations as at 31 January 2021 ", "Registrations_2021") \
                               .withColumnRenamed(" Registrations as at 31 January 2022 ", "Registrations_2022") \
                               .withColumnRenamed(" Registrations as at 31 January 2023 ", "Registrations_2023")

# Convert columns to integers
EV_df = EV_df.withColumn("Registrations_2021", 
                                           col("Registrations_2021").cast("int")) \
                               .withColumn("Registrations_2022", 
                                           col("Registrations_2022").cast("int")) \
                               .withColumn("Registrations_2023", 
                                           col("Registrations_2023").cast("int"))

# Show the DataFrame
EV_df.show()

+--------+-----+-----------+------------------+------------------+------------------+
|Postcode|State|  Fuel Type|Registrations_2021|Registrations_2022|Registrations_2023|
+--------+-----+-----------+------------------+------------------+------------------+
|     800|   NT|        ICE|              NULL|              NULL|              NULL|
|     800|   NT|        BEV|                 3|                 3|                12|
|     800|   NT|      HFCEV|              NULL|              NULL|              NULL|
|     800|   NT|Hybrid/PHEV|               131|               411|               402|
|     810|   NT|        ICE|              NULL|              NULL|              NULL|
|     810|   NT|        BEV|                 6|                16|                22|
|     810|   NT|      HFCEV|              NULL|              NULL|              NULL|
|     810|   NT|Hybrid/PHEV|               288|               375|               467|
|     812|   NT|        ICE|              NULL|       

### Merging Vehicle registration dataset with Polulation-Income-Postcode dataset

In [20]:
# Perform the join operation
merged_df = Pop_Inc_PC_df.join(EV_df, on='Postcode', how='inner')

# Show the resulting DataFrame
merged_df.show()


+--------+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+-----------+------------+-----+-----------+------------------+------------------+------------------+
|Postcode|LGA code|Local Government Area|Population in 2021|Population in 2022|Population in 2023|Number of Earners|Median Age of Earners|Mean Income|Lat_precise|Long_precise|State|  Fuel Type|Registrations_2021|Registrations_2022|Registrations_2023|
+--------+--------+---------------------+------------------+------------------+------------------+-----------------+---------------------+-----------+-----------+------------+-----+-----------+------------------+------------------+------------------+
|    3691|   10050|               Albury|             56067|             56665|             57517|            31816|                  43 |      62395|   -36.1241|     146.882|  VIC|Hybrid/PHEV|                44|                73|               1

In [21]:
# Convert PySpark DataFrame to Pandas DataFrame
merged_pd_df = merged_df.toPandas()

### Saving the cleaned dataset to CSV

In [22]:
# Write merged dataframe to CSV
merged_pd_df.to_csv("../cleaned_data/Population_Income_Registration.csv")